# Проверка качества работы алгоритмов машинного обучения
## Введение
В процессе работы с моделями машинного обучения важной задачей является оценка качества их работы. В данной главе мы разберем, как работать с имеющимися данными, чтобы получить представление о том, насколько точно модель предсказывает значения новых данных, не предъявлявшихся алгоритму в процессе обучения. Кроме того, будут рассмотрены метрики, позволяющие оценить точность предсказания (классификации). 
Оценка качества алгоритмов машинного обучения с помощью перекрестной проверки (ресемплинга)
Для понимания того, насколько можно доверять модели машинного обучения в процессе эксплуатации, необходимо провести проверку того, насколько хорошо выполняется предсказание данных, незнакомых модели, не использовавшихся для обучения. Лучший способ оценить качество работы алгоритма – это получить прогноз для записей, правильный ответ для которых известен, но которые не были включены в обучающую выборку. Второй способ – использовать статистические методы, называемые методами повторной выборки или ресемплинга.
## Оценка алгоритмов машинного обучения 
Почему нельзя обучить модель машинного обучения на имеющемся наборе данных и использовать прогнозы из этого же набора данных для оценки качества получившейся модели? Простой ответ заключается в термине «переобучение».
Представьте себе алгоритм, который запоминает каждое наблюдение, предъявленное ему во время обучения. В случае если оценка будет производиться на том же наборе данных, который использовался для обучения алгоритма, хорошо обученная модель получает идеальную оценку, но при этом благодаря избыточной сложности модели прогнозы, сделанные ею на основе новых данных, будут неточными, такая модель обладает большой памятью, но малой обобщающей способностью. Таким образом, модель машинного обучения необходимо тестировать на данных, которые не использовались для её обучения.
Оценка модели машинного обучения представляет собой меру, которую мы можем использовать, чтобы оценить, насколько хорошо алгоритм может работать на практике. Высокие значения оценок, к сожалению, не являются гарантией высокого качества модели. С накоплением большего набора данных необходимо постоянно производить оценку качества работы модели и, при необходимости, дообучать модель на большем наборе данных, обеспечив тем самым качество модели в процессе эксплуатации. В разделе будут рассмотрены четыре способа разделения набора данных и создания оценок качества работы модели машинного обучения:
- Обучающие и тестовые наборы.
- k-кратная перекрестная проверка.
- Исключите одну перекрестную проверку.
- Повторные случайные разбиения выборки данных на обучающие и тестовые наборы.

## Разделение данных на обучающие и тестовые наборы
Самый простой метод, который мы можем использовать для оценки качества модели машинного обучения это использование различных наборов данных для обучения и тестирования. Исходный набор данных делится на две части, а затем модель обучается на первой части, а прогноз выполняется на на второй части, после чего выполняется оценка соответствия прогноза ожидаемым результатам. Пропорция, в которой производится разделение, может зависеть от размера и специфики вашего набора данных, хотя обычно используется 67% данных для обучения и оставшиеся 33% для тестирования.
Этот метод оценки алгоритма достаточно простой технически и быстрый по времени выполнения. Он идеально подходит для больших наборов данных (миллионы записей), где есть убедительные доказательства того, что оба подмножества данных репрезентативны для главной задачи проблемы. Высокая скорость работы алгоритма позволяет эффективно использовать его совместно с трудоемкими алгоритмами машинного обучения. Недостатком этого метода является то, что он может иметь высокую дисперсию, то есть отклонения между учебным и тестовыми наборами данных могут привести к значимым отклонениям в оценке точности. В приведенном ниже примере мы разбиваем набор данных индейцев Пима на 67%/33% для обучения, тестирования и оценки точности модели логистической регрессии.

In [1]:
# Оценка алгоритма с помощью обучающей и тестовой выборок
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
filename = 'pima-indians-diabetes.csv'
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = read_csv(filename, names=names)
array = dataframe.values
X = array[:,0:8]
Y = array[:,8]
test_size = 0.33
max_iter = 200
seed = 7
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
model = LogisticRegression(max_iter=max_iter)
model.fit(X_train, Y_train)
result = model.score(X_test, Y_test)
print("Общая точность: %.3f%%" % (result*100.0))

Общая точность: 78.740%


Можно видеть, что расчетная точность модели составила примерно 78%. Обратите внимание, что помимо пропорции распределения данных на обучающие и тестовые, мы также указываем случайное начальное значение. Поскольку разделение данных случайно, необходимо гарантировать воспроизводимость результатов. Указывая случайное начальное значение, мы гарантируем, что получаем одни и те же случайные числа при каждом запуске кода и, в свою очередь, такое же разделение данных. Это важно с той точки зрения, что необходимо иметь возможность сравнить полученный результат с предполагаемой точностью другого алгоритма машинного обучения или того же алгоритма с другой конфигурацией. Чтобы убедиться, что сравнение было точным, необходимо гарантировать, что они обучены и протестированы на одних и тех же данных. 
## K-кратная кросс-валидация
Перекрестная проверка (говорят также «кросс-валидация») представляет собой подход оценки качества алгоритма машинного обучения, обеспечивающий меньшую дисперсию, чем при разделении данных на обучающую и тестовую часть. Суть подхода состоит в разделения набора данных на k частей (например, k = 5 или k = 10). Каждое разделение данных называется срезом (split). Алгоритм обучается на k-1 срезах, а оставшийся срез исключается из обучения и затем на нем тестируется и оценивается обученная модель. Процедура повторяется для до тех пор, пока через нее не пройдет каждый из k срезов исходных данных. Результатом выполнения k перекрестных проверок будет k различных оценок качества модели, которые можно суммировать, используя среднее значение и стандартное отклонение.
Результатом является более надежная оценка качества работы выбранной модели на новых данных. Точность оценки повышается за счет того, что модель обучается и оценивается несколько раз на разных данных. Выбор k должен обеспечивать достаточный размер тестовой выборки при каждой оценки, при этом позволяя выполнить достаточное количество повторений оценки модели, обученной различных подмножествах исходных данных, чтобы обеспечить справедливую оценку качества работы модели на новых данных. Для наборов данных небольшого размера, состоящих из тысяч или десятков тысяч записей, обычно используются значения k, равные 3, 5 и 10. В приведенном ниже примере используется 10-кратная перекрестная проверка

In [2]:
# Оценки алгоритма с k-кратной перекрестной проверкой
from pandas import read_csv
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
filename = 'pima-indians-diabetes.csv'
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = read_csv(filename, names=names)
array = dataframe.values
X = array[:,0:8]
Y = array[:,8]
num_folds = 10
seed = 7
max_iter = 200
kfold = KFold(n_splits=num_folds, random_state=seed, shuffle=True)
model = LogisticRegression(max_iter=max_iter)
results = cross_val_score(model, X, Y, cv=kfold)
print("Общая точность: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100.0))

Общая точность: 77.216% (4.968%)


Видно, что результатом оценки является как средние оценки качества модели, так и их дисперсия. При обобщении показателей качества работы модели рекомендуется суммировать распределение показателей, в данном случае предполагая гауссовское распределение (в большинстве случаев справедливое предположение) и фиксируя результаты для совокупности оценок по каждому из k испытаний срезов данных.
## Поэлементная кросс-валидация (LOOCV)
Перекрестная проверка может быть настроена таким образом, чтобы размер среза был равен 1 (k равно количеству наблюдений в вашем наборе данных). Этот вариант перекрестной проверки называется поэлементной перекрестной кросс-валидацией (Leave-One-Out Cross Validation, LOOCV). Результатом является большое количество показателей производительности, которые можно суммировать, чтобы дать более обоснованную оценку точности вашей модели на новых данных.
Недостатком является то, что это может быть более дорогостоящая в вычислительном отношении процедура, чем k-кратная кросс-валидация. В приведенном ниже примере используется поэлементная кросс-валидация.

In [3]:
# Оценка модели с помощью поэлементной кросс-валидации
from pandas import read_csv
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
max_iter = 500
filename = 'pima-indians-diabetes.csv'
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = read_csv(filename, names=names)
array = dataframe.values
X = array[:,0:8]
Y = array[:,8]
num_folds = 10
loocv = LeaveOneOut()
model = LogisticRegression(max_iter=max_iter)
results = cross_val_score(model, X, Y, cv=loocv)
print("Общая точность: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100.0))

Общая точность: 77.604% (41.689%)


Можно видеть, насколько выросла дисперсия результатов испытаний в поэлементной кросс-валидации по сравнению с k-кратной.
## Повторяющиеся случайные разбиения данных на обучающие и тестовые
Другой вариант k-кратной кросс-валидации заключается в создании случайного разделения данных, подобного описанному выше разделению обучающие/тестовые, но процесс повторяется несколько раз, при этом разбиение каждый раз выполняется заново. Такой подход обеспечивает сочетание скорости, свойственной методу разделения выборки на обучающую текстовую со снижением различий в качестве моделей, свойственное k-кратной кросс-валидации. Процесс может повторяться много раз по мере необходимости для достижения требуемой точности прогноза. Недостатком является то, что итерации могут включать в себя значительную часть одних и тех же данных в последовательности или разделять тест от запуска к запуску, что вносит избыточности в оценку. Приведенный ниже пример разбивает данные на 67%/33% для обучения/тестирования.

In [4]:
# Алгоритм кросс-валидации с многократным разделением выборки в случайном порядке
from pandas import read_csv
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
max_iter = 500
filename = 'pima-indians-diabetes.csv'
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = read_csv(filename, names=names)
array = dataframe.values
X = array[:,0:8]
Y = array[:,8]
n_splits = 10
test_size = 0.33
seed = 7
kfold = ShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=seed)
model = LogisticRegression(max_iter = max_iter)
results = cross_val_score(model, X, Y, cv=kfold)
print("Общая точность: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100.0))

Общая точность: 76.535% (2.235%)


Мы видим, что в этом случае распределение показателя качества модели находится на одном уровне с k-кратной перекрестной проверкой, выполненной в примере выше.

# Метрики качества алгоритмов машинного обучения
Задача выбора метрик, используемых для оценки качества моделей машинного обучения, имеет чрезвычайную важность. Выбор метрик определяет, как измеряется и сравнивается точность и производительность алгоритмов машинного обучения. Они влияют на то, как вы оцениваете важность различных характеристик в результатах и на ваш окончательный выбор того, какую модель выбрать.
Метрики оценки
В данном разделе будут рассмотрены различные метрики оценки качества моделей обучения, как для задач классификации, так и для задач регрессии. В каждом примере набор данных загружается непосредственно из репозитория машинного обучения в сети Интернет.
- Для показателей классификации в качестве демонстрации используется набор данных о начале диабета у индейцев Pima. Это задача двоичной классификации, в которой все входные переменные являются числовыми.
- Для регрессионных показателей в качестве демонстрации используется набор данных о ценах на жилье в Бостоне. Это задача регрессии, в которой все входные переменные также являются числовыми.

Все примеры включают оценку одной и той же модели: логистическую регрессию для классификации и линейную регрессию для регрессионных задач. Для демонстрации каждой метрики используется 10-кратный тест перекрестной валидации, поскольку это наиболее вероятный сценарий, который вы будете использовать при использовании различных метрик оценки моделей.
Во всех примерах раздела для отчета о качестве каждой из моделей используется функция кросс-валидации cross_validation.cross_val_score. Она позволяет использовать различные метрики оценки качества моделей. Для единообразия все оценки отображаются отсортированными в порядке возрастания (наибольший балл - лучший). Для того, чтобы не нарушать эту логику сортировки некоторые метрики оценки (например, среднеквадратичная ошибка), для которых наименьшее значение является наилучшим, выводятся функцией cross_val_score в виде отрицательных значений. Это важно помнить в ходе рассмотрения примеров, потому что некоторые метрики принимают отрицательные значения, которые возможны (как, например, сумма квадратов отклонений) только в случае использования комплексной арифметики.
## Метрики качества для задач классификации 
Задачи классификации, пожалуй, являются наиболее распространенными в машинном обучении, и поэтому существует множество метрик, которые можно использовать для оценки качества решения задач такого типа. В этом разделе мы рассмотрим, как использовать следующие метрики:
- Точность классификации (accuracy).
- Логарифмическая функция ошибки.
- Площадь под кривой ROC.
- Матрица ошибок (confusion matrix).
- Отчет о классификации.

## Метрика точности классификации (accuracy)

Точность классификации – это количество правильных предсказаний, отнесенное к числу всех сделанных прогнозов. Это наиболее распространенная метрика оценки для задач классификации, но именно с ее применением связано и наибольшее количество ошибок в применении. Следует помнить, что данная метрика дает точную оценку качества только тогда, когда обучающая выборка сбалансирована, то есть в ней представлено равное количество наблюдений для каждого класса (что случается редко) и что все ответы модели одинаково важны, что, наоборот, зачастую не соответствует действительности. Ниже приведен пример расчета точности классификации.

In [5]:
# Оценки модели по метрике точности классификации
from pandas import read_csv
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
max_iter = 1000
filename = 'pima-indians-diabetes.csv'
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = read_csv(filename, names=names)
array = dataframe.values
X = array[:,0:8]
Y = array[:,8]
kfold = KFold(n_splits=10, random_state=7, shuffle=True)
model = LogisticRegression(max_iter = max_iter)
scoring = 'accuracy'
results = cross_val_score(model, X, Y, cv=kfold, scoring=scoring)
print("Общая точность (Accuracy): %.3f (%.3f)" % (results.mean(), results.std()))

Общая точность (Accuracy): 0.772 (0.050)


Функция выдает соотношение в виде коэффициента, лежащего в диапазоне [0, 1]. Для лучшей понимаемости его можно преобразовать в процент правильных ответов модели, умножив значение на 100, что даст оценку точности примерно 77%.

## Логарифмическая функция ошибки

Логарифмическая функция ошибки (Logarithmic Loss или logloss) это метрика качества классификационной модели оценки предсказаний вероятностей принадлежности к данному классу. Скалярную вероятность между 0 и 1 можно рассматривать как меру уверенности в предсказании с помощью модели. Прогнозы, которые являются правильными или неправильными, соответственно вознаграждаются или наказываются пропорционально достоверности прогноза. Ниже приведен пример расчета logloss для предсказаний модели логистической регрессии в наборе данных о начале диабета у индейцев Пима.

In [6]:
# Оценка алгоритма по logloss
from pandas import read_csv
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
max_iter = 200
filename = 'pima-indians-diabetes.csv'
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = read_csv(filename, names=names)
array = dataframe.values
X = array[:,0:8]
Y = array[:,8]
kfold = KFold(n_splits=10, random_state=7, shuffle=True)
model = LogisticRegression()
scoring = 'neg_log_loss'
results = cross_val_score(model, X, Y, cv=kfold, scoring=scoring)
print("Логарифмическая оценка точности (Logloss): %.3f (%.3f)" % (results.mean(), results.std()))

Логарифмическая оценка точности (Logloss): -0.495 (0.060)


D:\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
D:\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_resu

Чем ниже оценка, тем лучше модель справляется с классификацией новых данных, в идеале (при полном отсутствии ошибок на тестовой выборке) данное значение должно равняться нулю. Как упоминалось выше, мера инвертируется, чтобы быть восходящей при использовании функции cross_val_score( ). 

## Площадь под кривой ROC

Площадь под ROC-кривой (Area Under Curve – площадь под кривой, Receiver Operating Characteristic – рабочих характеристика приёмника) – это метрика оценки для задач бинарной классификации (Binary Classification). Площадь под кривой (AUC) является мерой способности классификатора различать классы. Оценка, приближающаяся значению 1.0 соответствует модели, которая сделала все прогнозы идеально. Значение метрики в районе 0,5 характеризует модель, выдающую случайные значения класса.
Оценку ROC можно разбить на чувствительность и специфичность. Следует отметить, что решение задачи бинарной классификации постоянно связано с нахождением компромисса между такими показателями модели, как чувствительность и специфичность:
- Чувствительность (ее еще называют «отзыв») характеризуется долей истинно положительных (True Positive, TP) ответов модели. Это количество экземпляров из положительного класса, которые на самом деле были предсказаны правильно.
- Специфичность характеризуется долей истинно отрицательных (True Negative, TN) ответов модели, количеством экземпляров из отрицательного класса, которые были фактически предсказаны правильно.

Приведенный ниже пример демонстрирует расчет AUC.

In [7]:
# Оценка модели по AUC
from pandas import read_csv
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
max_iter = 200
filename = 'pima-indians-diabetes.csv'
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = read_csv(filename, names=names)
array = dataframe.values
X = array[:,0:8]
Y = array[:,8]
kfold = KFold(n_splits=10, random_state=7, shuffle=True)
model = LogisticRegression(max_iter = max_iter)
scoring = 'roc_auc'
results = cross_val_score(model, X, Y, cv=kfold, scoring=scoring)
print("Значение метрики AUC: %.3f (%.3f)" % (results.mean(), results.std()))

Значение метрики AUC: 0.829 (0.047)


Значение AUC, относительно близкое к 1 и превышающее 0,5 говорит о приемлемом уровне качества классификации.
## Матрица ошибок
Матрица ошибок (Confusion Matrix) представляет собой удобный инструмент оценки точности модели классификации на два или более классов. Рассмотрим идею матрицы ошибок подробнее, поскольку это один из фундаментальных инструментов оценки алгоритмов классификации, использующийся не только в машинном обучении но и, например, в доказательной медицине.
Ниже приведен пример вычисления матрицы ошибок для набора прогнозов логистической регрессии о начале диабета у индейцев Пима.
- TP - true-positive
- FP - false-positive
- TN - true-negative
- FN - false-negative

In [8]:
# Оценки модели по матрице ошибок
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
max_iter = 200
filename = 'pima-indians-diabetes.csv'
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = read_csv(filename, names=names)
array = dataframe.values
X = array[:,0:8]
Y = array[:,8]
test_size = 0.33
seed = 7
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
model = LogisticRegression(max_iter = max_iter)
model.fit(X_train, Y_train)
predicted = model.predict(X_test)
matrix = confusion_matrix(Y_test, predicted)
print("Матрица ощибок:")
print(matrix)

Матрица ощибок:
[[142  20]
 [ 34  58]]


Хотя массив напечатан без заголовков, вы можете видеть, что большинство прогнозов приходится на главную диагональ матрицы, соответствующую правильным предсказаниям – TP и TN.
## Отчет о классификации
Библиотека scikit-learn предоставляет удобный отчет о качестве решения моделью задачи классификации, позволяющий охарактеризовать качество модели. Функция classification_report() отображает показатели точности (precision), чувствительности (recall), оценки F1 (F1-score) и поддержки (support) для каждого класса. Приведенный ниже пример демонстрирует отчет по задаче бинарной классификации.

In [9]:
# Оценки модели классификации по классическому отчету
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
max_iter = 200
filename = 'pima-indians-diabetes.csv'
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = read_csv(filename, names=names)
array = dataframe.values
X = array[:,0:8]
Y = array[:,8]
test_size = 0.33
seed = 7
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
model = LogisticRegression(max_iter = max_iter)
model.fit(X_train, Y_train)
predicted = model.predict(X_test)
report = classification_report(Y_test, predicted)
print("Отчет по качеству классификации:")
print(report)

Отчет по качеству классификации:
              precision    recall  f1-score   support

         0.0       0.81      0.88      0.84       162
         1.0       0.74      0.63      0.68        92

    accuracy                           0.79       254
   macro avg       0.78      0.75      0.76       254
weighted avg       0.78      0.79      0.78       254



# Метрики качества для регрессионных моделей
В этом разделе будут рассмотрены 3 наиболее распространенные метрики для оценки прогнозов по регрессионным задачам машинного обучения:
- Средняя абсолютная погрешность.
- Среднеквадратичная ошибка.
- Коэффициент детерминации (критерий R^2).

## Средняя абсолютная погрешность
Средняя абсолютная погрешность (mean absolute error, MAE) – это сумма модулей разности между предсказаниями и фактическими значениями, отнесенная к числу оцениваемых примеров. Метрика позволяет оценить, насколько в среднем ошибочными были сформированные моделью прогнозы. Метрика дает представление о величине ошибки, но не дает представления о направлении (в плюс ошибалась модель или в минус). Приведенный ниже пример демонстрирует применение метрики средней абсолютной ошибки на прогнозе набора данных о ценах на жилье в Бостоне (загружается напрямую с сайта).

In [10]:
# Оценка модели регрессии по средней абсолютной ошибке
from pandas import read_csv
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
filename = 'housing.csv'
names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
dataframe = read_csv(filename, delim_whitespace=True, names=names)
# peek = dataframe.head(5)
# print(peek)
array = dataframe.values
X = array[:,0:13]
Y = array[:,13]
num_folds = 10
kfold = KFold(n_splits=10, random_state=7, shuffle=True)
model = LinearRegression()
scoring = 'neg_mean_absolute_error'
results = cross_val_score(model, X, Y, cv=kfold, scoring=scoring)
print("MAE: %.3f (%.3f)" % (results.mean(), results.std()))

MAE: -3.387 (0.667)


Значение 0 указывает на отсутствие ошибок или на идеальный прогноз. Как и logloss, эта метрика инвертируется функцией cross_val_score( ) для сохранения порядка сортировки результатов от лучших к худшим.
## Среднеквадратичная ошибка
Среднеквадратичная ошибка (mean square error, MSE) очень похожа на среднюю абсолютную погрешность в том смысле, что она дает общее представление о величине ошибки. Взятие квадратного корня из средней квадратичной ошибки преобразует масштаб обратно в естественную шкалу выходной переменной, что может быть значимым для представления модели специалистам предметной области, такая метрика называется. Такая метрика называется корнем из среднеквадратичной ошибки (root mean squared error, RMSE). Приведенный ниже пример демонстрирует вычисление средней квадратичной ошибки.

In [11]:
# Оценка модели регрессии по среднеквадратичной ошибке
from pandas import read_csv
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
filename = 'housing.csv'
names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
dataframe = read_csv(filename, delim_whitespace=True, names=names)
# peek = dataframe.head(5)
# print(peek)
array = dataframe.values
X = array[:,0:13]
Y = array[:,13]
num_folds = 10
kfold = KFold(n_splits=10, random_state=7, shuffle=True)
model = LinearRegression()
scoring = 'neg_mean_squared_error'
results = cross_val_score(model, X, Y, cv=kfold, scoring=scoring)
print("MSE: %.3f (%.3f)" % (results.mean(), results.std()))

MSE: -23.747 (11.143)


Эта метрика также инвертируется для обеспечения сортировки от лучших значений к худшим. Если вы не хотите получить комплексное значение (или сообщение в программе об арифметической ошибке), то не стоит забывать взять модуль от значения MSE перед расчетом RMSE.
## Коэффициент детерминации (критерий R^2)
Метрика R^2 (коэффициент детерминации) рассчитывается как доля дисперсии зависимой переменной, объясняемая рассматриваемой регрессионной моделью в зависимости входных переменных. Коэффициент детерминации принимает значения от 0 до 1, чем ближе значение коэффициента к 1, тем сильнее зависимость. При оценке регрессионных моделей это интерпретируется как соответствие модели данным. Для приемлемых моделей предполагается, что коэффициент детерминации должен быть хотя бы не меньше 50 % (в этом случае коэффициент множественной корреляции превышает по модулю 70 %). Модели с коэффициентом детерминации выше 80 % можно признать достаточно хорошими (коэффициент корреляции превышает 90 %). Значение коэффициента детерминации 1 означает функциональную зависимость между переменными.

In [12]:
# Оценка качества модели по коэффициенту детерминации
from pandas import read_csv
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
filename = 'housing.csv'
names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO',
'B', 'LSTAT', 'MEDV']
dataframe = read_csv(filename, delim_whitespace=True, names=names)
array = dataframe.values
X = array[:,0:13]
Y = array[:,13]
kfold = KFold(n_splits=10, random_state=7, shuffle=True)
model = LinearRegression()
scoring = 'r2'
results = cross_val_score(model, X, Y, cv=kfold, scoring=scoring)
print("R^2: %.3f (%.3f)" % (results.mean(), results.std()))

R^2: 0.718 (0.099)


Видно, что прогнозы модели имеют неплохое соответствие фактическим значениям, поскольку значение R^2 несколько выше 0,7.